# Algorithm optimization & testing

This notebook demonstrates performance testing for three different implementations of an algorithm which removes even numbers from a list.

## Notebook set-up

In [1]:
import time
import random
import numpy as np
import matplotlib.pyplot as plt
from memory_profiler import memory_usage

## 1. Algorithm implementations

In [2]:
# Three functions to remove even numbers: one that uses
# a loop, one that uses Numpy and one that uses a list comprehension.

def loop_remove_evens(nums: list) -> list:
    '''Removes even numbers from a list by looping over the
    list, checking each element and collecting the odd ones.'''

    evens=[]

    for num in nums:
        if num %2 != 0:
            evens.append(num)

    return evens


def numpy_remove_evens(nums: list) -> list:
    '''Removes even numbers from list using NumPy.'''

    nums=np.array(nums)

    return nums[nums % 2 != 0]


def list_comp_remove_evens(nums: list) -> list:
    '''Removes even numbers from list using list comprehension.'''

    return [x for x in nums if x % 2 !=0]


nums=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
print(f'Loop results: {loop_remove_evens(nums)}')
print(f'Numpy result: {numpy_remove_evens(nums)}')
print(f'List comprehension result: {list_comp_remove_evens(nums)}')

Loop results: [1, 3, 5, 7, 9]
Numpy result: [1 3 5 7 9]
List comprehension result: [1, 3, 5, 7, 9]


The three functions give the same result - but are they equivalent?

## 2. Algorithm implementation testing

In [3]:
n=500       # Number of trials to run
k=10000000  # Number of random integers to use

### 2.1. Memory footprint measurement

In [ ]:
# Run the memory usage test with n replicates of k numbers each
loop_memory_use=[]
numpy_memory_use=[]
list_comprehension_memory_use=[]

# Collet n memory use measurements with different random lists
for trial in range(n):

    # Generate some random integers for this trial
    random_list=random.choices(list(range(0,100)), k=k)

    # Test loop version
    mem_usage=memory_usage((loop_remove_evens, (random_list,)))
    loop_memory_use.append(max(mem_usage))

    # Test numpy version
    mem_usage=memory_usage((numpy_remove_evens, (random_list,)))
    numpy_memory_use.append(max(mem_usage))

    # Test list comprehension version
    mem_usage=memory_usage((list_comp_remove_evens, (random_list,)))
    list_comprehension_memory_use.append(max(mem_usage))

### 2.2. Execution time measurement

In [ ]:
# Run execution time test with n replicates of k numbers each
loop_execution_time=[]
numpy_execution_time=[]
list_comprehension_execution_time=[]

# Collet n speed measurements with different random lists
for trial in range(n):

    # Generate some random integers for this trial
    random_list=random.choices(list(range(0,100)), k=k)

    # Test loop version
    start=time.time()
    loop_remove_evens(random_list)
    loop_execution_time.append(time.time() - start)

    # Test numpy version
    start=time.time()
    numpy_remove_evens(random_list)
    numpy_execution_time.append(time.time() - start)

    # Test list comprehension version
    start=time.time()
    list_comp_remove_evens(random_list)
    list_comprehension_execution_time.append(time.time() - start)

## 3. Results

In [ ]:
# Plot the results
import matplotlib.pyplot as plt

fig, axs=plt.subplots(1, 2, figsize=(8, 4))
axs=axs.flatten()

fig.suptitle(f'Even number removal performance results\n(n={n}, k={k})')

_, memory_bins=np.histogram(loop_memory_use + numpy_memory_use + list_comprehension_memory_use, bins=20)

axs[0].set_title('Memory footprint')
axs[0].hist(loop_memory_use, bins=memory_bins, alpha=0.3, label='Loop')
axs[0].hist(numpy_memory_use, bins=memory_bins, alpha=0.3, label='Numpy')
axs[0].hist(list_comprehension_memory_use, bins=memory_bins, alpha=0.3, label='List comprehension')
axs[0].set_xlabel('Memory footprint (MB)')
axs[0].set_ylabel('Trials')
axs[0].set_yscale('log')
axs[0].legend(loc='best')

_, time_bins=np.histogram(loop_execution_time + numpy_execution_time + list_comprehension_execution_time, bins=20)

axs[1].set_title('Execution time')
axs[1].hist(loop_execution_time, bins=time_bins, alpha=0.3, label='Loop')
axs[1].hist(numpy_execution_time, bins=time_bins, alpha=0.3, label='Numpy')
axs[1].hist(list_comprehension_execution_time, bins=time_bins, alpha=0.3, label='List comprehension')
axs[1].set_xlabel('Execution time (sec.)')
axs[1].set_ylabel('Trials')
axs[1].set_yscale('log')
axs[1].legend(loc='best')

plt.tight_layout()
plt.show()